# Music Recommender Lab 3 : SageMaker Pipelines

----
In this final notebook, we'll combine all the steps we've gone over in each individual notebook, and condense them down into a [SageMaker Pipelines](https://docs.aws.amazon.com/sagemaker/latest/dg/pipelines.html) object which will automate  the entire modeling process from the beginning of data ingestion to monitoring the model. SageMaker Pipelines is a tool for building machine learning pipelines that take advantage of direct SageMaker integration. Because of this integration, you can create a pipeline and set up SageMaker Projects for orchestration using a tool that handles much of the step creation and management for you.

----
### Contents
- [Overview](00_overview_arch_data.ipynb)
- [Part 1: Data Prep using SageMaker Processing](01_music_rec_data_prep.ipynb)
- [Part 2: Model Training and Hyperparameter Tuning](02_music_rec_model_training.ipynb)
- [Part 3: SageMaker Pipelines](03_music_rec_pipelines.ipynb)
    - [Architecture](#06-arch)
    - [Pipelines Overview](#pipelines)

## SageMaker Pipelines

SageMaker Pipelines supports the following activities, which are demonstrated in this notebook:

* Pipelines - A DAG of steps and conditions to orchestrate SageMaker jobs and resource creation.
* Processing job steps - A simplified, managed experience on SageMaker to run data processing workloads, such as feature engineering, data validation, model evaluation, and model interpretation.
* Training job steps - An iterative process that teaches a model to make predictions by presenting examples from a training dataset.
* Conditional execution steps - A step that provides conditional execution of branches in a pipeline.
* Register model steps - A step that creates a model package resource in the Model Registry that can be used to create deployable models in Amazon SageMaker.
* Create model steps - A step that creates a model for use in transform steps or later publication as an endpoint.
* Transform job steps - A batch transform to preprocess datasets to remove noise or bias that interferes with training or inference from a dataset, get inferences from large datasets, and run inference when a persistent endpoint is not needed.
* Parametrized Pipeline executions - Enables variation in pipeline executions according to specified parameters.

## A SageMaker Pipeline

The pipeline that you create follows a typical machine learning (ML) application pattern of preprocessing, training, evaluation, model creation, batch transformation, and model registration:

![A typical ML Application pipeline](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-full.png)

### Install required and/or update third-party libraries

In [ ]:
%pip install -Uq pip
%pip install -q sagemaker==2.98.0 awswrangler


## Preparation


Let's start by specifying:

- The S3 bucket and prefix that you want to use for training and model data.  This should be within the same region as the Notebook Instance, training, and hosting.
- The IAM role arn used to give training and hosting access to your data. See the documentation for how to create these.  Note, if more than one role is required for notebook instances, training, and/or hosting, please replace the boto regexp with a the appropriate full IAM role arn string(s).


In [ ]:
import json
import boto3
import pathlib
import sagemaker
import numpy as np
import pandas as pd
import awswrangler as wr
import time

from sagemaker import get_execution_role
import sys
import pprint
sys.path.insert(1, './code')

from sagemaker.estimator import Estimator
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.debugger import Rule, rule_configs
from sagemaker.inputs import TrainingInput
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker.model_metrics import (
    MetricsSource,
    ModelMetrics,
)
from sagemaker.processing import (
    ProcessingInput,
    ProcessingOutput,
    Processor,
    ScriptProcessor,
)
from sagemaker.processing import FrameworkProcessor
from sagemaker.xgboost import XGBoostProcessor
from sagemaker.s3 import S3Uploader
from sagemaker.workflow.conditions import ConditionLessThanOrEqualTo
from sagemaker.workflow.condition_step import ConditionStep
from sagemaker.workflow.functions import JsonGet

from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)
from sagemaker.workflow.pipeline import Pipeline
from sagemaker.workflow.properties import PropertyFile
from sagemaker.workflow.steps import (
    ProcessingStep,
    TrainingStep,
    CacheConfig,
)
from sagemaker.workflow.step_collections import RegisterModel
from sagemaker.utils import name_from_base
from sagemaker.xgboost import XGBoost

from sagemaker.workflow.model_step import ModelStep
from sagemaker import Model
from sagemaker.workflow.retry import (
    StepRetryPolicy,
    StepExceptionTypeEnum,
    SageMakerJobExceptionTypeEnum,
    SageMakerJobStepRetryPolicy,
)

from sagemaker.transformer import Transformer
from sagemaker.inputs import TransformInput
from sagemaker.workflow.steps import TransformStep

from sagemaker.lineage.visualizer import LineageTableVisualizer

In [ ]:
sagemaker.__version__

### Set region and boto3 config

In [ ]:
# most of the time there's no need to import boto3 directly, everything is already in sagemaker, and less confusing for data scientists
session = sagemaker.Session()
sm_client = session.sagemaker_client
role = get_execution_role()
bucket = session.default_bucket()
region = session.boto_region_name
pipeline_session = sagemaker.workflow.pipeline_context.PipelineSession()

In [ ]:
prefix = 'music-recommendation-workshop'
base_job_prefix = 'music-rec-workshop'

<a id='pipelines'></a>

## SageMaker Pipeline Overview

##### [back to top](#06-nb)

---- 

#### [Step 1: SageMaker Preprocessing Step](#sagemaker-processing)
#### [Step 2: Train XGboost Model](#pipe-train-xgb)
#### [Step 3: Model Evaluation](#pipe-eval)
#### [Step 4: Register Model](#pipe-Register-Model)
#### [Step 5: Batch Transform](#deploy)
#### [Combine Steps and Run Pipeline](#combine)

Now that you've manually done each step in our machine learning workflow, you can certain steps to allow for faster model experimentation without sacrificing transparncy and model tracking. In this section you will create a pipeline which trains a new model, persists the model in SageMaker and then adds the model to the registry.


### Define Parameters to Parametrize Pipeline Execution

Define Pipeline parameters that you can use to parametrize the pipeline. Parameters enable custom pipeline executions and schedules without having to modify the Pipeline definition.

The supported parameter types include:

* `ParameterString` - represents a `str` Python type
* `ParameterInteger` - represents an `int` Python type
* `ParameterFloat` - represents a `float` Python type

These parameters support providing a default value, which can be overridden on pipeline execution. The default value specified should be an instance of the type of the parameter.

The parameters defined in this workflow include:

* `processing_instance_type` - The `ml.*` instance type of the processing job.
* `processing_instance_count` - The instance count of the processing job.
* `training_instance_type` - The `ml.*` instance type of the training job.
* `model_approval_status` - What approval status to register the trained model with for CI/CD purposes ( "PendingManualApproval" is the default).
* `tracks_data` - The S3 bucket URI location of the input tracks data
* `ratings_data` - The S3 bucket URI location of the input ratings data

![Define Parameters](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-1.png)

In [ ]:

process_script = 'create_datasets.py'
    
tracks_data = f's3://{bucket}/{prefix}/input/tracks.csv'
ratings_data = f's3://{bucket}/{prefix}/input/ratings.csv'

# parameters for pipeline execution
     
input_tracks = ParameterString(
    name="InputTracksUrl",
    default_value=tracks_data,
)
input_ratings = ParameterString(
    name="InputRatingsUrl",
    default_value=ratings_data,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount", default_value=1
)
training_instance_type = ParameterString(
    name="TrainingInstanceType", default_value="ml.m5.xlarge"
)
model_approval_status = ParameterString(
    name="ModelApprovalStatus", default_value="PendingManualApproval"
)
model_output = ParameterString(
    name="ModelOutputUrl",
    default_value=f"s3://{bucket}/{prefix}/model",
)


<a id='sagemaker-processing'></a>
### Step 1: SageMaker Processing Step
### Define a Processing Step for Feature Engineering

First, develop a preprocessing script that is specified in the Processing step.

In the SageMaker Processing lab, we have already created the `preprocess.py`, which contains the preprocessing script, and stored the path to the script. You can use the following cell to load the stored value.

The Processing step executes the script on the input data. The Training step uses the preprocessed training features and labels to train a model. The Evaluation step uses the trained model and preprocessed test features and labels to evaluate the model.

[Pipeline Overview](#pipelines)


![Define a Processing Step for Feature Engineering](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-2.png)

Create an instance of an `SKLearnProcessor` processor and use that in our `ProcessingStep`.

You also specify the `framework_version` to use throughout this notebook.

Note the `processing_instance_type` and `processing_instance_count` parameters used by the processor instance.

In [ ]:
# processing step for feature engineering
est_cls = SKLearn
framework_version_str="0.20.0"

base_job_name = 'sm-music-processing'
sklearn_processor = FrameworkProcessor(
    #----------------------------------------------------------
    # TODO - fill in the processor parameters
    # Please fill in this section of code by referring the previous notebooks
    #----------------------------------------------------------
)

processing_job_name = name_from_base(f"{base_job_prefix}-preprocess")
train_path = f"s3://{bucket}/{prefix}/{processing_job_name}/train"
val_path = f"s3://{bucket}/{prefix}/{processing_job_name}/val"
test_path = f"s3://{bucket}/{prefix}/{processing_job_name}/test"


step_args = sklearn_processor.run(
    outputs=[
        ProcessingOutput(output_name="train", source="/opt/ml/processing/output/train", destination=train_path),
        ProcessingOutput(output_name="validation", source="/opt/ml/processing/output/val", destination=val_path),
        ProcessingOutput(output_name="test", source="/opt/ml/processing/output/test", destination=test_path),
    ],
    code=process_script,
    source_dir='code',
    arguments=[
                 '--base_dir', '/opt/ml/processing',
    ],
    inputs=[
        ProcessingInput(
            source=input_tracks,
            destination="/opt/ml/processing/input/tracks",
            s3_data_distribution_type="FullyReplicated",
        ),
        ProcessingInput(
            source=input_ratings,
            destination="/opt/ml/processing/input/ratings",
            s3_data_distribution_type="FullyReplicated",
        ),
    ],
)


step_process = ProcessingStep(
   name="PreprocessData",
   step_args=step_args,
)

Next, use the processor instance to construct a `ProcessingStep`, along with the input and output channels, and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Note the `input_data` parameters passed into `ProcessingStep` is the input data used in the step. This input data is used by the processor instance when it is run.

Also, note the `"train"` and `"test"` named channels specified in the output configuration for the processing job. Step `Properties` can be used in subsequent steps and resolve to their runtime values at execution. Specifically, this usage is called out when you define the training step.

<a id='pipe-train-xgb'></a>
### Define a Training Step to Train a Model

In this section, use Amazon SageMaker's [XGBoost Algorithm](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html) to train on this dataset. Configure an Estimator for the XGBoost algorithm and the input dataset. A typical training script loads data from the input channels, configures training with hyperparameters, trains a model, and saves a model to `model_output` so that it can be hosted later. 

The model path where the models from training will be saved is also specified.

Note the `training_instance_type` parameter may be used in multiple places in the pipeline. In this case, the `training_instance_type` is passed into the estimator.

![Define a Training Step to Train a Model](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-3.png)

In [ ]:
# training step for generating model artifacts
hyperparameters = {"num_round": 6, "K": 5}

estimator_parameters = {
    #----------------------------------------------------------
    # TODO - fill in training job param
    # Please fill in this section of code by referring the previous notebooks
    #----------------------------------------------------------
}

xgb_train = XGBoost(**estimator_parameters, 
                    sagemaker_session=pipeline_session)


Finally, use the estimator instance to construct a `TrainingStep` as well as the `properties` of the prior `ProcessingStep` used as input in the `TrainingStep` inputs and the code that's executed when the pipeline invokes the pipeline execution. This is similar to an estimator's `fit` method in the Python SDK.

Pass in the `S3Uri` of the `"train"` output channel to the `TrainingStep`. Also, use the other `"validation"` output channel for model evaluation in the pipeline. The `properties` attribute of a Pipeline step matches the object model of the corresponding response of a describe call. These properties can be referenced as placeholder values and are resolved at runtime. For example, the `ProcessingStep` `properties` attribute matches the object model of the [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response object.

In [ ]:
step_args = xgb_train.fit(inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
        "validation": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "validation"
            ].S3Output.S3Uri,
            content_type="text/csv",
        ),
    },)

step_train = TrainingStep(
    name="TrainModel",
    step_args=step_args,
)

<a id='pipe-eval'></a>
### Define a Model Evaluation Step to Evaluate the Trained Model

First, develop an evaluation script that is specified in a Processing step that performs the model evaluation.

After pipeline execution, you can examine the resulting `evaluation.json` for analysis.

The evaluation script uses `xgboost` to do the following:

* Load the model.
* Read the test data.
* Issue predictions against the test data.
* Build a classification report, including mae, mse, rmse and r2 metrics.
* Save the evaluation report to the evaluation directory.

![Define a Model Evaluation Step to Evaluate the Trained Model](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-4.png)

Create an instance of a `ScriptProcessor` processor and use it in the `ProcessingStep`.

Note the `processing_instance_type` parameter passed into the processor.

In [ ]:
# processing step for evaluation
script_eval = XGBoostProcessor(
    framework_version='1.5-1',
    sagemaker_session=pipeline_session,
    role=role,
    instance_type='ml.m5.xlarge',
    instance_count=1,
    base_job_name=f"{base_job_prefix}/script-eval",
)


Use the processor instance to construct a `ProcessingStep`, along with the input and output channels and the code that will be executed when the pipeline invokes pipeline execution. This is similar to a processor instance's `run` method in the Python SDK.

Specifically, the `S3ModelArtifacts` from the `step_train` `properties` and the `S3Uri` of the `"test"` output channel of the `step_process` `properties` are passed into the inputs. The `TrainingStep` and `ProcessingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) and  [DescribeProcessingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeProcessingJob.html) response objects, respectively.

In [ ]:
evaluation_report = PropertyFile(
    name="EvaluationReport",
    output_name="evaluation",
    path="evaluation.json",
)

step_args = script_eval.run(
    outputs=[
        ProcessingOutput(
            output_name="evaluation", source="/opt/ml/processing/evaluation"
        ),
    ],
    inputs=[
        ProcessingInput(
            source=step_train.properties.ModelArtifacts.S3ModelArtifacts,
            destination="/opt/ml/processing/model",
        ),
        ProcessingInput(
            source=step_process.properties.ProcessingOutputConfig.Outputs[
                "test"
            ].S3Output.S3Uri,
            destination="/opt/ml/processing/test",
        ),
    ],
    code="evaluation.py",
    source_dir="code",
)

step_eval = ProcessingStep(
    name="EvaluateModel",
    step_args=step_args,
    property_files=[evaluation_report],
)

### Define a Create Model Step to Create a Model

In order to perform batch transformation using the example model, create a SageMaker model. 

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

![Define a Create Model Step and Batch Transform to Process Data in Batch at Scale](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-5.png)

In [ ]:
model = Model(
    image_uri=xgb_train.image_uri,
    model_data=step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=pipeline_session,
    role=role,
)

step_create_model = ModelStep(
    name="MusicRecomCreateModel",
    step_args=model.create(instance_type="ml.m5.xlarge"),
    retry_policies=[
            StepRetryPolicy(exception_types=[StepExceptionTypeEnum.THROTTLING], max_attempts=3)
        ]
)


<a id='deploy'></a>
### Define a Transform Step to Perform Batch Transformation

Now that a model instance is defined, create a `Transformer` instance with the appropriate model type, compute instance type, and desired output S3 URI.

Specifically, pass in the `ModelName` from the `CreateModelStep`, `step_create_model` properties. The `CreateModelStep` `properties` attribute matches the object model of the [DescribeModel](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeModel.html) response object.

In [ ]:
batch_output = f"s3://{bucket}/{prefix}/batchtransform"
transformer = Transformer(
    model_name=step_create_model.properties.ModelName,
    instance_type="ml.m5.xlarge",
    instance_count=1,
    output_path=batch_output,
    accept="text/csv",
    assemble_with="Line",
    sagemaker_session=pipeline_session,
)

Pass in the transformer instance and the `TransformInput` with the `batch_data` pipeline parameter defined earlier. More details of the `TransformInput` can be found [here](https://github.com/aws/sagemaker-python-sdk/blob/2594ffb3eaefaf55936b71ea0c38442135223602/src/sagemaker/inputs.py#L140). We use `input_filter` to filter out the first column, which is the label column, in the test data. We also set the batch job to join the input source with the batch job output. Please check the document about [associating inference with input record](https://docs.aws.amazon.com/sagemaker/latest/dg/batch-transform-data-processing.html) for more details

In [ ]:
step_args = transformer.transform(
            #----------------------------------------------------------
            # TODO - fill in the transform parameters for pipeline
            # Please fill in this section of code by referring the previous notebooks
            #----------------------------------------------------------
        )
step_transform = TransformStep(
        name="MusicRecomTransform",
        step_args=step_args,
    )

<a id='pipe-Register-Model'></a>
### Define a Register Model Step to Create a Model Package

Use the estimator instance specified in the training step to construct an instance of `RegisterModel`. The result of executing `RegisterModel` in a pipeline is a model package. A model package is a reusable model artifacts abstraction that packages all ingredients required for inference. Primarily, it consists of an inference specification that defines the inference image to use along with an optional model weights location.

A model package group is a collection of model packages. A model package group can be created for a specific ML business problem, and new versions of the model packages can be added to it. Typically, customers are expected to create a ModelPackageGroup for a SageMaker pipeline so that model package versions can be added to the group for every SageMaker Pipeline run.

The construction of `RegisterModel` is similar to an estimator instance's `register` method in the Python SDK.

Specifically, pass in the `S3ModelArtifacts` from the `TrainingStep`, `step_train` properties. The `TrainingStep` `properties` attribute matches the object model of the [DescribeTrainingJob](https://docs.aws.amazon.com/sagemaker/latest/APIReference/API_DescribeTrainingJob.html) response object.

Note that the specific model package group name provided in this notebook can be used in the model registry and CI/CD work with SageMaker Projects.

In [ ]:
# register the model if evaluation score is satisfactory
model_package_group_name = "MusicRecomModelPackage"

model_metrics = ModelMetrics(
    model_statistics=MetricsSource(
        s3_uri="{}/evaluation.json".format(
            step_eval.arguments["ProcessingOutputConfig"]["Outputs"][0]["S3Output"][
                "S3Uri"
            ]
        ),
        content_type="application/json",
    )
)

register_arg = model.register(
    content_types=["text/csv"],
    response_types=["text/csv"],
    inference_instances=["ml.t2.medium", "ml.m5.large"],
    transform_instances=["ml.m5.xlarge"],
    model_package_group_name=model_package_group_name,
    approval_status=model_approval_status,
    model_metrics=model_metrics,
)
step_register = ModelStep(
    name="RegisterModel",
    step_args=register_arg,
)


### Define a Condition Step to Check Accuracy and Conditionally Create a Model and Run a Batch Transformation and Register a Model in the Model Registry

In this step, the model is registered only if the accuracy of the model, as determined by the evaluation step `step_eval`, exceeded a specified value. A `ConditionStep` enables pipelines to support conditional execution in the pipeline DAG based on the conditions of the step properties. 

In the following section, you:

* Define a `ConditionLessThanOrEqualTo` on the accuracy value found in the output of the evaluation step, `step_eval`.
* Use the condition in the list of conditions in a `ConditionStep`.
* Pass the `CreateModelStep` and `TransformStep` steps, and the `RegisterModel` step collection into the `if_steps` of the `ConditionStep`, which are only executed, if the condition evaluates to `True`.

![Define a Condition Step to Check Accuracy and Conditionally Execute Steps](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-6.png)

In [ ]:
# condition step for evaluating model quality and branching execution
cond_lte = ConditionLessThanOrEqualTo(
    left=JsonGet(
        step_name=step_eval.name,
        property_file=evaluation_report,
        json_path="regression_metrics.rmse.value",
    ),
    right=1.2,
)

step_cond = ConditionStep(
    name="CheckEvaluation",
    conditions=[cond_lte],
    if_steps=[step_register, step_create_model, step_transform],
    else_steps=[],
)


<a id='combine'></a>
### Define a Pipeline of Parameters, Steps, and Conditions

In this section, combine the steps into a Pipeline so it can be executed.

A pipeline requires a `name`, `parameters`, and `steps`. Names must be unique within an `(account, region)` pair.

Note:

* All of the parameters used in the definitions must be present.
* Steps passed into the pipeline do not have to be listed in the order of execution. The SageMaker Pipeline service resolves the _data dependency_ DAG as steps for the execution to complete.
* Steps must be unique to across the pipeline step list and all condition step if/else lists.

![Define a Pipeline of Parameters, Steps, and Conditions](https://raw.githubusercontent.com/aws/amazon-sagemaker-examples/a78c7255cbe7892408d8ea2b15a7a2117703befc/sagemaker-pipelines/tabular/abalone_build_train_deploy/img/pipeline-7.png)

In [ ]:
# pipeline instance
pipeline = Pipeline(
    name="music-recommedation-workshop-pipeline-0822",
    parameters=[
        input_tracks,
        input_ratings,
        processing_instance_count,
        training_instance_type,
        model_approval_status,
        model_output,
    ],
    #----------------------------------------------------------
    # TODO - fill in the steps parameter of pipeline
    # Please fill in this section of code by referring the previous notebooks
    #----------------------------------------------------------
    sagemaker_session=pipeline_session,
)

#### (Optional) Examining the pipeline definition

The JSON of the pipeline definition can be examined to confirm the pipeline is well-defined and the parameters and step properties resolve correctly.

In [ ]:
# import json


# definition = json.loads(pipeline.definition())
# definition

### Submit the pipeline to SageMaker and start execution

Submit the pipeline definition to the Pipeline service. The role passed in will be used by the Pipeline service to create all the jobs defined in the steps.

In [ ]:
pipeline.upsert(role_arn=role)

In [ ]:
execution = pipeline.start()

In [ ]:
execution.describe()

In [ ]:
# Wait for the execution to complete.
execution.wait()

### Lineage

Review the lineage of the artifacts generated by the pipeline.

In [ ]:
viz = LineageTableVisualizer(sagemaker.session.Session())
for execution_step in reversed(execution.list_steps()):
    print(execution_step)
    display(viz.show(pipeline_execution_step=execution_step))
    time.sleep(5)

### Parametrized Executions

You can run additional executions of the pipeline and specify different pipeline parameters. The parameters argument is a dictionary containing parameter names, and where the values are used to override the defaults values.

Based on the performance of the model, you might want to kick off another pipeline execution on a compute-optimized instance type and set the model approval status to "Approved" automatically. This means that the model package version generated by the `RegisterModel` step is automatically ready for deployment through CI/CD pipelines, such as with SageMaker Projects.

In [ ]:
execution = pipeline.start(
    parameters=dict(
        TrainingInstanceType="ml.c5.xlarge",
        ModelApprovalStatus="Approved",
    )
)

In [ ]:
execution.wait()
execution.list_steps()